In [1]:
# ======================
# Imports
# ======================
import math as mt
import scipy.optimize as spo


# ======================
# Scenario Setup
# ======================
# Given data
S1 = 421e3          # Load 1 = 421 kW
pf1 = 0.80           # Power factor (lagging)
S2 = 317e3          # Load 2 = 317 kW
pf2 = 0.77           # Power factor (lagging)
Z_line = 0.95 + 2.47j    # Line impedance (ohms)
V_source = 49e3      # Source RMS voltage (volts)

# Compute power angles
theta1 = mt.acos(pf1)
theta2 = mt.acos(pf2)

# Complex powers for both loads (S = P + jQ)
S_load1 = S1 * (mt.cos(theta1) + 1j * mt.sin(theta1))
S_load2 = S2 * (mt.cos(theta2) + 1j * mt.sin(theta2))


# ======================
# Secant Method Function
# ======================
def f(V):
    """
    Residual function f(V) = |V + I*Z_line| - V_source
    where I = (S_total.conjugate()) / V
    """
    S_total = S_load1 + S_load2
    I = S_total.conjugate() / V
    V_send = V + I * Z_line
    return abs(V_send) - V_source


# ======================
# Secant Method Setup
# ======================
X0 = 30000       # Initial guess (30 kV)
X1 = 49000       # Second initial guess (49 kV)
MAX_ITERS = 100  # Iteration limit
X_TOL = 1e-7     # Tolerance

# Run Secant Method using scipy's newton (without derivative)
p_sc, p_sc_info = spo.newton(
    f, X0, fprime=None, args=(),
    tol=X_TOL, maxiter=MAX_ITERS,
    full_output=True, x1=X1
)


# ======================
# Results
# ======================
print(f"Finding a root from {{X0}}:")
print(p_sc_info)
print(f"Residual value: {{f(p_sc)}}")
print(f"The RMS voltage at the common bus is {{p_sc / 1000:.4f}} kilovolts.")

Finding a root from {X0}:
      converged: True
           flag: converged
 function_calls: 5
     iterations: 4
           root: 48965.78102180318
         method: secant
Residual value: {f(p_sc)}
The RMS voltage at the common bus is {p_sc / 1000:.4f} kilovolts.
